In [ ]:
# default_exp ought

# Ought Text Classification Project

> Binary classification on scientific paper abstracts

In [ ]:
#hide
from nbdev.showdoc import *

## GPT-2 Prompt Manipulation

The simplest approach is to to feed in a small number of trainging examples into the prompt directly for zero-shot classification.

### Setup

In [ ]:
#export
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import json

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval().cuda()

### Utilities

In [ ]:
#export
def generate(prompt, max_length=5, stop_token=None):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    generated_text_ids = model.generate(input_ids=input_ids.cuda(), max_length=max_length+len(input_ids[0]), do_sample=False)
    generated_text = tokenizer.decode(generated_text_ids[0], clean_up_tokenization_spaces=True)
    post_prompt_text = generated_text[len(tokenizer.decode(input_ids[0], clean_up_tokenization_spaces=True)):]
    return prompt + post_prompt_text[:post_prompt_text.find(stop_token) if stop_token else None]

In [ ]:
# Note that the logits are shifted over 1 to the left, since HuggingFace doesn't give a logit for the first token
def get_logits_and_tokens(text):
    input_ids = tokenizer.encode(text, return_tensors="pt")
    tokens = [tokenizer.decode([input_id]) for input_id in input_ids[0]]
    output = model(input_ids.cuda())
    return output.logits[0][:-1], tokens

In [ ]:
EXAMPLE_PROMPT = """Horrible: negative
Great: positive
Bad:"""

generated_text = generate(EXAMPLE_PROMPT, stop_token="\n")
generated_text

In [ ]:
logits, tokens = get_logits_and_tokens(generated_text)
last_token_probs = torch.softmax(logits[-1], dim=0)
negative_prob = last_token_probs[tokenizer.encode(" negative")[0]]
positive_prob = last_token_probs[tokenizer.encode(" positive")[0]]

print(f"tokens: {tokens}\nnegative prob: {negative_prob}\npositive prob: {positive_prob}")

### Load Data

Define helper function to load text from `.jsonl` files.

In [ ]:
#export
def load_jsonl(filename):
    f = open(filename)
    return [json.loads(line) for line in f.read().splitlines()]

In [ ]:
train_examples = load_jsonl("data/train.jsonl")
train_examples[-1]

In [ ]:
#export
def render_example(example):
    title = example["text"].split(".")[0].strip()
    abstract = example["text"][len(title)+1:].strip()
    return f"""Title: {title}
Abstract: {abstract}
Label: {"AI" if example["label"] == "True" else "Not AI"}"""

In [ ]:
#export
def render_end_example(example):
    title = example["text"].split(".")[0].strip()
    abstract = example["text"][len(title)+1:].strip()
    return f"""Title: {title}
Abstract: {abstract}
Label:"""

In [ ]:
#export
def make_prompt(instructions, train_examples, end_example):
    rendered_train_examples = "\n\n--\n\n".join([render_example(example) for example in train_examples])
    return f"""{instructions}

{rendered_train_examples}

--

{render_end_example(end_example)}"""

In [ ]:
INSTRUCTIONS = "Classify the following examples based on whether they are AI-relevant or not:"

prompt = make_prompt(INSTRUCTIONS, train_examples[:4], train_examples[4])
print(prompt)

In [ ]:
generated_text = generate(prompt, stop_token="\n")
print(generated_text)

## Training from Scratch

Prompt engineering can only get you so far, and is not guaranteed to generate valid labels. It _should_ be possible to retrain a network using a small number of examples in under a minute.

In [ ]:
from fastai.text.all import *